In [1]:
!pip install imageio-ffmpeg torch torchvision

In [2]:
import os
import sys
import logging
import tempfile
import shutil
import subprocess
import traceback
import gc
from typing import List, Set, Dict, Optional, Any
from dataclasses import dataclass, field
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import threading
from dataclasses import asdict
from sagemaker.pytorch import PyTorchProcessor
from sagemaker import get_execution_role
from sagemaker.processing import ProcessingOutput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
mp.set_start_method("spawn", force=True) 

_PIPELINE = None
_PIPELINE_DEVICE = None

In [4]:
# Import torch first to avoid registration conflicts
import torch
import torchvision  # Import this explicitly before transformers

# Env setup
import os
import imageio_ffmpeg, os
os.environ["PATH"] = os.path.dirname(imageio_ffmpeg.get_ffmpeg_exe()) + os.pathsep + os.environ.get("PATH","")
os.environ.setdefault("TRANSFORMERS_CACHE", "/tmp/transformers_cache")
os.environ.setdefault("HF_HOME", "/tmp/hf_home")
os.environ.setdefault("TORCH_HOME", "/tmp/torch_home")

import boto3
from botocore.exceptions import ClientError
import pandas as pd
from tqdm import tqdm
import imageio_ffmpeg
from transformers import pipeline
import argparse, os, logging

/opt/conda/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-07 03:43:19.711098: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 03:43:19.729589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759808599.750165   30653 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759808599.757035   30653 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to regis

In [5]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
    force = True,
)
logger = logging.getLogger("asr_pipeline")

In [6]:
# =============================================== Config ===============================================
@dataclass
class Config:
    """Configuration for the transcription pipeline"""
    # S3 settings
    s3_input: str = "s3://asrelder-data/common_voice/23/cv-corpus-23.0-2025-09-05/en/clips/"
    output_local_csv: str = "./transcripts_from_prefix.csv"
    write_back_to_s3: bool = False
    output_s3_uri: Optional[str] = None
    #output_s3_uri: str = "s3://asrelder-data/outputs/transcripts_from_prefix.csv" uncomment if we want it back on s3
    validation_csv_path: str = "common_voices_23_train_with_validated_votes.csv"
    validation_csv_column: str = "path"

    # Processing settings
    max_files: Optional[int] = None
    download_workers: int = 24
    append_every_n: int = 200
    resume_from_csv: bool = True

    # Model settings
    model_id: str = "openai/whisper-base"
    language: Optional[str] = "en"
    task: str = "transcribe"
    chunk_length_s: int = 30
    stride_length_s: tuple = (5, 5)

    # File settings
    audio_extensions: List[str] = field(default_factory=lambda: [".mp3", ".wav", ".flac", ".m4a", ".ogg"])

In [7]:
def _build_worker_pipeline(cfg_dict, device_id):
    use_cuda = torch.cuda.is_available() and device_id >= 0
    dtype = torch.float16 if use_cuda else torch.float32
    generate_kwargs = {}
    if cfg_dict.get("language"): generate_kwargs["language"] = cfg_dict["language"]
    if cfg_dict.get("task"):     generate_kwargs["task"] = cfg_dict["task"]
    return pipeline(
        "automatic-speech-recognition",
        model=cfg_dict["model_id"],
        device=(device_id if use_cuda else -1),
        torch_dtype=dtype,
        return_timestamps=True,
        chunk_length_s=cfg_dict["chunk_length_s"],
        stride_length_s=tuple(cfg_dict["stride_length_s"]),
        generate_kwargs=(generate_kwargs or None),
    )

def _transcribe_worker(args):
    # args: (audio_path, cfg_dict, device_id)
    audio_path, cfg_dict, device_id = args
    global _PIPELINE, _PIPELINE_DEVICE
    if (_PIPELINE is None) or (_PIPELINE_DEVICE != device_id):
        _PIPELINE = _build_worker_pipeline(cfg_dict, device_id)
        _PIPELINE_DEVICE = device_id
    try:
        out = _PIPELINE(audio_path)
        text = out.get("text", "") if isinstance(out, dict) else str(out)
        return {"path": audio_path, "text": text, "error": ""}
    except Exception as e:
        return {"path": audio_path, "text": "", "error": f"{type(e).__name__}: {e}"}

In [8]:
# =============================================== Core Components ===============================================
class FFmpegSetup:
    """Manages FFmpeg availability"""

    @staticmethod
    def ensure_available() -> Optional[str]:
        """Check if ffmpeg is available on PATH"""
        ff = None
        try:
            ff = imageio_ffmpeg.get_ffmpeg_exe()
        except Exception as e:
            logger.warning(f"imageio-ffmpeg error: {e}")

        if ff and os.path.exists(ff):
            ff_dir = os.path.dirname(ff)
            os.environ["PATH"] = ff_dir + os.pathsep + os.environ.get("PATH", "")

        resolved = shutil.which("ffmpeg")
        if resolved:
            try:
                out = subprocess.run(
                    [resolved, "-version"],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    check=True,
                    timeout=5
                )
                logger.info(f"ffmpeg: {resolved} | {out.stdout.decode('utf-8').splitlines()[0]}")
            except Exception:
                logger.info(f"ffmpeg: {resolved} (version check failed)")
        else:
            # Create shim if needed
            if ff and os.path.exists(ff):
                bin_dir = os.path.expanduser("~/.local/bin")
                os.makedirs(bin_dir, exist_ok=True)
                shim = os.path.join(bin_dir, "ffmpeg")
                with open(shim, "w") as f:
                    f.write(f"#!/usr/bin/env bash\n\"{ff}\" \"$@\"\n")
                os.chmod(shim, 0o755)
                os.environ["PATH"] = bin_dir + os.pathsep + os.environ.get("PATH", "")
                resolved = shutil.which("ffmpeg")
                if resolved:
                    logger.info(f"ffmpeg shim created: {resolved}")

        if not resolved:
            logger.warning("FFmpeg not found; use torchaudio fallback")

        return resolved

class S3Manager:
    """Handles S3 operations"""

    def __init__(self, config: Config):
        self.config = config
        self.client = boto3.client("s3")

    def parse_uri(self, uri: str) -> tuple[str, str]:
        """Parse S3 URI into bucket and key"""
        if not uri.startswith("s3://"):
            raise ValueError(f"Invalid S3 URI: {uri}")
        p = urlparse(uri)
        return p.netloc, p.path.lstrip("/")

    def is_audio_file(self, key: str) -> bool:
        """Check if key is an audio file"""
        return any(key.lower().endswith(ext) for ext in self.config.audio_extensions)

    def list_audio_keys(self, bucket: str, prefix: str, allowed_filenames: Optional[Set[str]] = None) -> List[str]:
        """List all audio keys under prefix, optionally filtered by allowed filenames"""
        if self.is_audio_file(prefix):
            return [prefix]

        keys = []
        paginator = self.client.get_paginator("list_objects_v2")

        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for obj in page.get("Contents", []):
                key = obj["Key"]
                if not key.endswith("/") and self.is_audio_file(key):
                    if allowed_filenames:
                        filename = os.path.basename(key)
                        if filename not in allowed_filenames:
                            continue
                            
                    keys.append(key)
                    if self.config.max_files and len(keys) >= self.config.max_files:
                        return keys
        return keys

    def download_to_temp(self, bucket: str, key: str) -> str:
        """Download S3 object to temporary file"""
        _, ext = os.path.splitext(key)
        if not ext:
            ext = ".mp3"

        fd, tmp_path = tempfile.mkstemp(suffix=ext)
        os.close(fd)

        with open(tmp_path, "wb") as f:
            self.client.download_fileobj(bucket, key, f)

        return tmp_path

    def upload_file(self, local_path: str, s3_uri: str):
        """Upload file to S3"""
        bucket, key = self.parse_uri(s3_uri)
        self.client.upload_file(local_path, bucket, key)
        logger.info(f"Uploaded to {s3_uri}")

class TranscriptionManager:
    """Manages multiple ASR pipelines for parallel processing"""

    def __init__(self, config: Config):
        self.config = config
        self.pipelines = []
        self.pipeline_locks = []

        num_workers = getattr(config, 'asr_workers', 16)

        # Create multiple pipeline instances
        for i in range(num_workers):
            if torch.cuda.is_available():
                device_id = i % torch.cuda.device_count()
            else:
                device_id = -1

            logger.info(f"Creating ASR worker {i+1}/{num_workers} on device {device_id}")
            pipe = self._build_pipeline(device_id=device_id)
            self.pipelines.append(pipe)
            self.pipeline_locks.append(threading.Lock())

    def _build_pipeline(self, device_id=-1):
        """Build the ASR pipeline"""
        use_cuda = torch.cuda.is_available() and device_id >= 0
        dtype = torch.float16 if use_cuda else torch.float32

        generate_kwargs = {}
        if self.config.language:
            generate_kwargs["language"] = self.config.language
        if self.config.task:
            generate_kwargs["task"] = self.config.task

        logger.info(f"Loading ASR: {self.config.model_id} (device={device_id}, dtype={dtype})")

        return pipeline(
            "automatic-speech-recognition",
            model=self.config.model_id,
            device=device_id,
            torch_dtype=dtype,
            return_timestamps=True,
            chunk_length_s=self.config.chunk_length_s,
            stride_length_s=self.config.stride_length_s,
            generate_kwargs=generate_kwargs or None,
        )

    def transcribe(self, audio_path: str) -> Dict[str, Any]:
        """Single file transcription - uses first pipeline"""
        return self._transcribe_with_pipeline(audio_path, 0)

    def transcribe_batch_parallel(self, paths_and_keys: List[tuple]) -> List[Dict[str, Any]]:
        """Transcribe multiple files in parallel using ThreadPoolExecutor"""
        num_workers = len(self.pipelines)

        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            futures = []
            for i, (path, key) in enumerate(paths_and_keys):
                pipeline_idx = i % num_workers
                future = executor.submit(
                    self._transcribe_with_pipeline_and_key,
                    path,
                    key,
                    pipeline_idx
                )
                futures.append(future)

            # Collect results
            results = []
            for future in as_completed(futures):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    logger.error(f"Transcription failed: {e}")

            return results

    def _transcribe_with_pipeline(self, audio_path: str, pipeline_idx: int) -> Dict[str, Any]:
        """Transcribe using a specific pipeline instance"""
        with self.pipeline_locks[pipeline_idx]:
            try:
                result = self.pipelines[pipeline_idx](audio_path)
                text = result.get("text", "") if isinstance(result, dict) else str(result)
                return {"text": text, "error": None}
            except Exception as e:
                if "ffmpeg" in str(e).lower():
                    return self._fallback_transcribe(audio_path, pipeline_idx)
                return {"text": "", "error": f"{type(e).__name__}: {e}"}

    def _transcribe_with_pipeline_and_key(self, audio_path: str, s3_key: str, pipeline_idx: int) -> Dict[str, Any]:
        """Transcribe with key tracking for batch processing"""
        result = self._transcribe_with_pipeline(audio_path, pipeline_idx)

        if result["text"]:
            preview = result["text"][:100] + "..." if len(result["text"]) > 100 else result["text"]
            logger.info(f"✓ Worker {pipeline_idx}: {os.path.basename(s3_key)}: {preview}")
        else:
            logger.warning(f"✗ Worker {pipeline_idx}: {os.path.basename(s3_key)}: {result['error']}")

        return {
            "s3_key": s3_key,
            "filename": os.path.basename(s3_key),
            "transcribed_text": result["text"],
            "error": result["error"] or ""
        }

    def _fallback_transcribe(self, audio_path: str, pipeline_idx: int) -> Dict[str, Any]:
        """Fallback using torchaudio"""
        try:
            import torchaudio
            waveform, sr = torchaudio.load(audio_path)
            if waveform.ndim == 2:
                waveform = waveform.mean(dim=0, keepdim=True)

            with self.pipeline_locks[pipeline_idx]:
                result = self.pipelines[pipeline_idx](waveform.squeeze(0).numpy(), sampling_rate=sr)
                text = result.get("text", "") if isinstance(result, dict) else str(result)
                return {"text": text, "error": None}
        except Exception as e:
            return {"text": "", "error": f"Fallback failed: {e}"}

    def cleanup_gpu_memory(self):
        """Clean up GPU memory"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()


class CSVManager:
    """Handles CSV operations """

    def __init__(self, csv_path: str):
        self.csv_path = csv_path

    def read_processed_keys(self) -> Set[str]:
        """Read already processed S3 keys from CSV"""
        if not os.path.exists(self.csv_path):
            return set()
        try:
            df = pd.read_csv(self.csv_path, usecols=["s3_key"])
            return set(df["s3_key"].astype(str).tolist())
        except Exception as e:
            logger.warning(f"Could not read existing CSV: {e}")
            return set()

    def append_results(self, results: List[Dict[str, Any]]):
        """Append results to CSV"""
        if not results:
            return

        df = pd.DataFrame(results)
        mode = "a" if os.path.exists(self.csv_path) else "w"
        header = not os.path.exists(self.csv_path)

        df.to_csv(self.csv_path, index=False, mode=mode, header=header)
        logger.info(f"Appended {len(results)} rows to {self.csv_path}")

In [9]:
class ValidationCSVManager:
    """Handles the CSV with validated files"""

    def __init__(self, csv_path: str, column_name: str):
        self.csv_path = csv_path
        self.column_name = column_name

    def read_allowed_filenames(self) -> Set[str]:
        """Read the list of allowed filenames from the validation CSV"""
        if not os.path.exists(self.csv_path):
            raise FileNotFoundError(f"Validation CSV not found: {self.csv_path}")

        try:
            df = pd.read_csv(self.csv_path, usecols=[self.column_name])
            filenames = df[self.column_name].astype(str).tolist()
            # Remove any NaN values and strip whitespace
            filenames = [f.strip() for f in filenames if pd.notna(f) and f.strip()]
            logger.info(f"Loaded {len(filenames)} allowed filenames from {self.csv_path}")
            return set(filenames)
        except Exception as e:
            logger.error(f"Failed to read validation CSV: {e}")
            raise

In [10]:
class ProductionPipeline:
    """Main for the transcription pipeline"""

    def __init__(self, config: Config):
        self.config = config
        self.s3_manager = S3Manager(config)
        self.transcription_manager = TranscriptionManager(config)
        self.csv_manager = CSVManager(config.output_local_csv)
        self.validation_csv_manager = ValidationCSVManager(
            config.validation_csv_path,
            config.validation_csv_column
        )
        self.results_buffer: List[Dict[str, Any]] = []

    def run(self):
        """Execute the transcription pipeline"""
        # Setup
        FFmpegSetup.ensure_available()

        allowed_filenames = self.validation_csv_manager.read_allowed_filenames()
        logger.info(f"Will only process files from validation CSV: {len(allowed_filenames)} files")


        bucket, prefix = self.s3_manager.parse_uri(self.config.s3_input)
        prefix_norm = prefix.rstrip("/") + "/"

        # Normalize allow-list to just basenames so it matches S3 keys
        def _norm(name: str) -> str:
            name = (name or "").strip().lstrip("/")
            return os.path.basename(name)

        allowed_filenames = { _norm(x) for x in self.validation_csv_manager.read_allowed_filenames() }
        logger.info(f"Will only process files from validation CSV (normalized): {len(allowed_filenames)}")


        existing = []
        paginator = self.s3_manager.client.get_paginator("list_objects_v2")
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix_norm):
            for obj in page.get("Contents", []):
                key = obj["Key"]
                if key.endswith("/"):
                    continue
                if not self.s3_manager.is_audio_file(key):
                    continue
                if os.path.basename(key) in allowed_filenames:
                    existing.append(key)

        all_keys = existing
        logger.info(f"Using {len(all_keys)} existing keys from CSV list (after intersection).")



        logger.info(f"Found {len(all_keys)} matching files in S3")

        processed_keys = set()
        if self.config.resume_from_csv:
            processed_keys = self.csv_manager.read_processed_keys()

        keys_to_process = [k for k in all_keys if k not in processed_keys]

        logger.info(
            f"Processing {len(keys_to_process)} files "
            f"(skipped {len(processed_keys)} already done)"
        )

        if not keys_to_process:
            logger.info("No files to process")
            return

        # Process with concurrent downloads
        self._process_with_concurrency(bucket, keys_to_process)

        # Final flush
        self._flush_results()

        # Upload to S3 if configured
        if self.config.write_back_to_s3 and self.config.output_s3_uri:
            self.s3_manager.upload_file(
                self.config.output_local_csv,
                self.config.output_s3_uri
            )

        logger.info("Pipeline complete!")


    def _process_with_concurrency(self, bucket: str, keys: List[str]):
        """Process files with concurrent downloads and parallel transcription"""
        batch_size = max(1, self.config.download_workers * 2)

        with ThreadPoolExecutor(max_workers=self.config.download_workers) as pool:
            for i in range(0, len(keys), batch_size):
                batch = keys[i:i + batch_size]

                # Download batch
                downloaded = []
                futures = {
                    pool.submit(self.s3_manager.download_to_temp, bucket, k): k
                    for k in batch
                }

                progress = tqdm(
                    as_completed(futures),
                    total=len(futures),
                    desc=f"Downloading batch {i//batch_size + 1}"
                )

                for future in progress:
                    key = futures[future]
                    try:
                        local_path = future.result()
                        downloaded.append((local_path, key))
                    except Exception as e:
                        logger.error(f"Download failed for {key}: {e}")
                        self.results_buffer.append({
                            "s3_key": key,
                            "filename": os.path.basename(key),
                            "transcribed_text": "",
                            "error": f"Download failed: {e}"
                        })

                # Transcribe downloaded files in parallel
                if downloaded:
                    logger.info(f"Transcribing {len(downloaded)} files in parallel...")
                    results = self.transcription_manager.transcribe_batch_parallel(downloaded)

                    for result in results:
                        self.results_buffer.append(result)

                    # Clean up temp files
                    for local_path, _ in downloaded:
                        try:
                            os.remove(local_path)
                        except:
                            pass

                # Periodic flush
                if len(self.results_buffer) >= self.config.append_every_n:
                    self._flush_results()

                # Memory cleanup
                self.transcription_manager.cleanup_gpu_memory()

    def _process_single_file(self, future, key: str):
        """Process a single downloaded file"""
        local_path = None
        try:
            # Get downloaded file
            local_path = future.result()

            # Transcribe
            result = self.transcription_manager.transcribe(local_path)

            # Store result
            self.results_buffer.append({
                "s3_key": key,
                "filename": os.path.basename(key),
                "transcribed_text": result["text"],
                "error": result["error"] or ""
            })

            if result["text"]:
                preview = result["text"][:100] + "..." if len(result["text"]) > 100 else result["text"]
                logger.info(f"{os.path.basename(key)}: {preview}")
            else:
                logger.warning(f"{os.path.basename(key)}: {result['error']}")

            # Periodic flush
            if len(self.results_buffer) >= self.config.append_every_n:
                self._flush_results()

            # Memory cleanup
            self.transcription_manager.cleanup_gpu_memory()

        except Exception as e:
            tb = traceback.format_exc(limit=2)
            self.results_buffer.append({
                "s3_key": key,
                "filename": os.path.basename(key),
                "transcribed_text": "",
                "error": f"{type(e).__name__}: {e} | {tb}"
            })
            logger.error(f"Failed to process {key}: {e}")

        finally:
            # Clean up temp file
            if local_path and os.path.exists(local_path):
                try:
                    os.remove(local_path)
                except Exception:
                    pass

    def _flush_results(self):
        """Flush results buffer to CSV"""
        if self.results_buffer:
            self.csv_manager.append_results(self.results_buffer)
            self.results_buffer.clear()

In [11]:
# ======================================== Run main loop ========================================
def main():
    try:
        config = Config()
        pipeline = ProductionPipeline(config)
        pipeline.run()
        return 0
    except KeyboardInterrupt:
        logger.info("Interrupted by user")
        return 130
    except Exception as e:
        logger.error(f"Pipeline failed: {e}")
        logger.error(traceback.format_exc())
        return 1


if __name__ == "__main__":
    result = main()
    if result == 0:
        print("Pipeline completed successfully!")
    else:
        print(f"Pipeline failed with code: {result}")

2025-10-07 03:43:25,997 INFO - Creating ASR worker 1/16 on device 0
2025-10-07 03:43:25,998 INFO - Loading ASR: openai/whisper-base (device=0, dtype=torch.float16)


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


2025-10-07 03:43:27,384 INFO - Creating ASR worker 2/16 on device 0
2025-10-07 03:43:27,384 INFO - Loading ASR: openai/whisper-base (device=0, dtype=torch.float16)


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


2025-10-07 03:43:27,905 INFO - Creating ASR worker 3/16 on device 0
2025-10-07 03:43:27,905 INFO - Loading ASR: openai/whisper-base (device=0, dtype=torch.float16)


...

2025-10-07 04:11:49,340 INFO - ✓ Worker 7: common_voice_en_18460460.mp3:  You take what?
2025-10-07 04:11:49,380 INFO - ✓ Worker 13: common_voice_en_18460454.mp3:  The same noise.
2025-10-07 04:11:49,559 INFO - ✓ Worker 10: common_voice_en_18460449.mp3:  Nor can I imagine.
2025-10-07 04:11:49,937 INFO - ✓ Worker 9: common_voice_en_18460453.mp3:  I have no complaints to make.
2025-10-07 04:11:50,047 INFO - ✓ Worker 3: common_voice_en_18460233.mp3:  Those wistful eyes of hers.
2025-10-07 04:11:50,326 INFO - ✓ Worker 15: common_voice_en_18460551.mp3:  There was the animal loser.
2025-10-07 04:11:50,488 INFO - ✓ Worker 11: common_voice_en_18460733.mp3:  My wife sings in the local choral society.
2025-10-07 04:11:50,515 INFO - ✓ Worker 1: common_voice_en_18460246.mp3:  In the spring, the former served his seeds.
2025-10-07 04:11:50,612 INFO - ✓ Worker 6: common_voice_en_18460234.mp3:  White Leckhorn said Mrs. Mortimer.
2025-10-07 04:11:50,624 INFO - ✓ Worker 2: common_voice_en_18460240.mp3:

2025-10-07 04:11:59,994 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:11:59,995 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,003 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,006 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,006 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10


2025-10-07 04:12:00,075 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,076 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,094 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,095 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10
2025-10-07 04:12:00,098 WARNING - Connection pool is full, discarding connection: asrelder-data.s3.amazonaws.com. Connection pool size: 10


2025-10-07 04:12:00,100 INFO - Transcribing 48 files in parallel...


2025-10-07 04:12:03,135 INFO - ✓ Worker 6: common_voice_en_18465459.mp3:  Here is in critical condition.
2025-10-07 04:12:03,386 INFO - ✓ Worker 11: common_voice_en_18466081.mp3:  and the air was growing chilly.
2025-10-07 04:12:03,627 INFO - ✓ Worker 3: common_voice_en_18466092.mp3:  He made a weary jester.
2025-10-07 04:12:03,726 INFO - ✓ Worker 15: common_voice_en_18467067.mp3:  Yes, lovely, isn't it?
2025-10-07 04:12:03,924 INFO - ✓ Worker 14: common_voice_en_18466099.mp3:  Their prized collection of ornaments was stolen.
2025-10-07 04:12:04,214 INFO - ✓ Worker 4: common_voice_en_18465528.mp3:  The air in the hut was close and oppressive.
2025-10-07 04:12:04,441 INFO - ✓ Worker 9: common_voice_en_18466083.mp3:  There's nothing for bringing you from learning to play guitar.
2025-10-07 04:12:04,446 INFO - ✓ Worker 2: common_voice_en_18466098.mp3:  A club acquaintance and a mere one at that.
2025-10-07 04:12:04,547 INFO - ✓ Worker 13: common_voice_en_18465340.mp3:  Ask if she needs a 